In [92]:
import nltk
# from nltk.corpus.reader import ConllCorpusReader
# import nltk.chunk.named_entity
# from nltk.classify import MaxentClassifier
import pickle


from nltk.parse.corenlp import CoreNLPParser
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
# nltk.config_megam('../megam_i686.opt')

from itertools import chain

import random
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
import os
import matplotlib.pyplot as plt
import pandas as pd

In [21]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.de import German
import pycrfsuite
from spacy.tokenizer import Tokenizer

In [93]:
pos_tagger = CoreNLPParser('http://localhost:9002', tagtype='pos')
# in stanford-corenlp-full-2018-02-27 directory
#English server: java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
#-preload tokenize,ssplit,pos,lemma,ner,parse,depparse \
#-status_port 9000 -port 9000 -timeout 15000 & 
# German server: java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
#-serverProperties StanfordCoreNLP-german.properties \
#-preload tokenize,ssplit,pos,ner,parse \
#-status_port 9002  -port 9002 -timeout 15000
# ner_tagger = pickle.load(open('../models/conll2003ger', 'rb'))
anonymize_tag_list = ['PER', 'LOC']
model_index = 0
model_list = [('nltkconll', '../../models/nltkconllfinal'), ('crfjoint', '../../models/finaljoint'), ('crfconll', '../../models/finalconll')]

In [102]:


# for model in model_list:
ner_pii_anonymize(pickle.load(open('../../models/nltkconllfinal', 'rb')), model_index, anonymize_tag_list)

Anonymization for the model 0 has begun
Anonymized text exported to: ../../outputs/anon_testdata.anonymized
Number of total sentences: 3005
Number of inconsistant sentences: 643
Number of sentences actually used: 2362
true positive: 1522
true negative: 36636
false positive: 539
false negative: 367
Precision: 0.7384764677341097
Recall: 0.8057173107464267
F1: 0.7706329113924052


In [103]:
ner_pii_anonymize(pickle.load(open('../../models/conll2003ger', 'rb')), model_index, anonymize_tag_list)

Anonymization for the model 0 has begun
Anonymized text exported to: ../../outputs/anon_testdata.anonymized
Number of total sentences: 3005
Number of inconsistant sentences: 670
Number of sentences actually used: 2335
true positive: 1490
true negative: 36393
false positive: 397
false negative: 349
Precision: 0.789613142554319
Recall: 0.8102229472539424
F1: 0.7997852925389156


In [101]:


def ner_pii_anonymize(ner_tagger, model_index, anon_tag_list):
    print('Anonymization for the model ' + str(model_index) + ' has begun')
    anonymized_text = ""
#     filename = '../../data/conll2003/anon_testdata'.split('/')[-1]
#     new_filename = 'anon_testdata.anonymized'
    output_path = '../../outputs/anon_testdata.anonymized'
    with open('../../data/conll2003/anon_testdata') as t:
        lines = t.readlines()
    for line in lines:
        tokens = WordPunctTokenizer().tokenize(line)
        pos_tagged = pos_tagger.tag(tokens)
#         print(pos_tagged)
        ne_parsetree = ner_tagger.parse(pos_tagged)
        anon_sent = anonymize_sent_tree(ne_parsetree, anon_tag_list)
        newline = TreebankWordDetokenizer().detokenize(anon_sent)
        anonymized_text += newline + '\n'
    with open(output_path, 'w') as op:
        op.writelines(anonymized_text)
        print('Anonymized text exported to: ' + output_path)
        model_index += 1
#     return anonymized_text
    evaluate(output_path)
        
def anonymize_sent_tree(ner_tagged_tree, anon_tag_list):
    anno_res = [anonymize_token(tok, anon_tag_list) for tok in ner_tagged_tree]
#     anno_res = map(anonymize_token, ner_tagged_tree)
    return list(chain(*anno_res))
    
def anonymize_token(token, anon_tag_list):
    if isinstance(token, nltk.Tree):
        if token.label() in anon_tag_list:
            # add '<' + '>'
            return [token.label()] * len(token)
        else:
            return [t[0] for t in token]
    elif isinstance(token, tuple):
        return [token[0],]
    
def is_tag(token):
    return token == 'PER' or token == 'LOC'

def evaluate(output_path):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    gtsents = []
    anonsents = []
    with open('../../data/conll2003/anon_testdata.dt.gt') as groundtruth:
        gtlines = groundtruth.readlines()
    for line1 in gtlines:
        gttokens = WordPunctTokenizer().tokenize(line1)
        gtsents.append(gttokens)
    # print(gttokens[20])

    with open(output_path) as anonfile:
        anonlines = anonfile.readlines()
    for line2 in anonlines:
        anontokens = WordPunctTokenizer().tokenize(line2)
        anonsents.append(anontokens)
    # print(anontokens[20])
    print('Number of total sentences: ' + str(len(gtsents)))
    incon = 0
    for i in range(len(gtsents)):
            if len(gtsents[i]) != len(anonsents[i]): incon+=1
            else:
                for j in range(len(gtsents[i])):
                    gttok = gtsents[i][j]
#             print(gttok)
                    anontok = anonsents[i][j]
#                 print(anontok)
                    if is_tag(gttok) and is_tag(anontok): tp += 1
                    elif not is_tag(gttok) and is_tag(anontok): fp += 1
                    elif is_tag(gttok) and not is_tag(anontok): fn += 1
                    elif not is_tag(gttok) and not is_tag(anontok): tn += 1
    print('Number of inconsistant sentences: ' + str(incon))
    print('Number of sentences actually used: ' +  str(len(gtsents) - incon))
    print("true positive: " + str(tp))
    print("true negative: " + str(tn))
    print("false positive: " + str(fp))
    print("false negative: " + str(fn))

    precision = tp/ (tp + fp)
    recall = tp/ (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    # for i in range(len(gttokens)):
    #     print(len(gttokens[i]))
    #     print(len(anontokens[i]))

    # tok_g = one token from ground_trouth

    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F1: " + str(f1))
#         for node in ne_parsetree:
#             ner_tag = str(node)[1:4]
#             if isinstance(node, nltk.tree.Tree) and (ner_tag in anonymize_tag_list):
#                 anon = ""
#                 for childnode in node:
#                     anon += '<' + ner_tag + '>'
#                 index = ne_parsetree.index(node)
#                 tokens[index] = anon
#         newline = TreebankWordDetokenizer().detokenize(tokens)
#         anonymized_text += newline + '\n'
# #     with open(new_filename, "w") as nt:
# #         f.writelines(filtered_lines)
#     # ... for every ner_tag in anonymize_tag_list replace token with ner-tag
#     return anonymized_text


In [44]:
ner_pii_anonymizee(None, model_index, anonymize_tag_list)

Anonymization for the model 0 has begun
['', '', '', '']
['', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'B-PE', 'I-PE', '', '', '', '', '', '', '', '', '']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', 'B-PE', 'I-PE', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'B-LO', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', 'B-LO', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
['', '', '', '', 'B-PE', 'I-PE', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [43]:
# 用cv里的方法 把confusion matrix 画出来 按照specified anon定义自己算出来

def ner_pii_anonymizee(ner_tagger, model_index, anon_tag_list):
#     df = pd.DataFrame()
    nlp = German()
    tokenizer = Tokenizer(nlp.vocab)
    print('Anonymization for the model ' + str(model_index) + ' has begun')
    anonymized_text = ""
    filename = '../../data/conll2003/anon_testdata'.split('/')[-1]
#     new_filename = 'anon_testdata.anonymized'
    output_path = '../../outputs/anon_testdata.anonymized'
    with open('../../data/conll2003/anon_testdata') as t:
        lines = t.readlines()
        data = []
        for line in lines:
            tokens = tokenizer(line.lstrip())
            data.append(tokens)
        X_train = [sent2features(s) for s in data]
        tagger = pycrfsuite.Tagger()
        tagger.open('../../models/finaljoint')
        for piece in X_train:
            predicted = tagger.tag(piece)
            print(predicted)
        
        
#     for line in lines:
#         tokens = tokenizer(line.lstrip())
#         
#         outputs = []
#         outputs.append(tagger.tag(sent2features(tokens)))
#         for i in range(len(outputs)):
#             output = outputs[i]
            
#             if output == 'LOC' or output == 'I-LOC' or output == 'B-LOC': tokens[i] = 'LOC'
#             elif output == 'PER' or output == 'I-PER' or output == 'B-PER': tokens[i] = 'PER'
#         newline = tokens
#         df.append(newline)
#         anonymized_text += newline + '\n'
#     with open(output_path, 'w') as op:
#         df.to_csv(output_path)
#         op.writelines(anonymized_text)
#         print('Anonymized text exported to: ' + output_path)
#             print(tokens)
#             print(outputs)
#     return anonymized_text
#     evaluatee(output_path)

# 没必要export 在里头比就行    

def is_tage(token):
    return token.text == 'PER' or token.text == 'LOC'

def evaluatee(output_path):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    gtsents = []
    anonsents = []
    with open('../../data/conll2003/anon_testdata.gt') as groundtruth:
        gtlines = groundtruth.readlines()
    for line1 in gtlines:
        gttokens = tokenizer(line1.lstrip())
        with open(output_path) as anonfile:
            anonlines = anonfile.readlines()
        for line2 in anonlines:
            anontokens = tokenizer(line2.lstrip())
            for i in range(len(gttokens)):
                if is_tage(gttokens[i]) and is_tage(anontokens[i]): tp += 1
                elif not is_tage(gttokens[i]) and is_tage(anontokens[i]): fp += 1
                elif is_tage(gttokens[i]) and not is_tage(anontokens[i]): fn += 1
                elif not is_tage(gttokens[i]) and not is_tage(anontokens[i]): tn += 1
def word2features(train_sample, i):
    token = train_sample[i]
    word = token.text
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.pos='+token.pos_,
        'word.dep='+token.dep_,
        'word.is_stop=%s' %token.is_stop,
        'word.lemma=' + token.lemma_,
        'word.tag=' + token.tag_,
        'word.shape=' + token.shape_,
        'word.is_alpha=%s' %token.is_alpha,        
    ]
    if i > 0:
        token1 = train_sample[i-1]
        word1 = token1.text
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.pos='+token1.pos_,
            '-1:word.dep='+token1.dep_,
            '-1:word.is_stop=%s' %token1.is_stop,
            '-1:word.lemma=' + token1.lemma_,
            '-1:word.tag=' + token1.tag_,
            '-1:word.shape=' + token1.shape_,
            '-1:word.is_alpha=%s' %token1.is_alpha,    
        ])
    else:
        features.append('BOS')
        
    if i < len(train_sample)-1:
        token1 = train_sample[i+1]
        word1 = token1.text
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.pos='+token1.pos_,
            '+1:word.dep='+token1.dep_,
            '+1:word.is_stop=%s' %token1.is_stop,
            '+1:word.lemma=' + token1.lemma_,
            '+1:word.tag=' + token1.tag_,
            '+1:word.shape=' + token1.shape_,
            '+1:word.is_alpha=%s' %token1.is_alpha,   
        ])
    else:
        features.append('EOS')       
    return features

def sent2features(train_sample):
    return [word2features(train_sample, i) for i in range(len(train_sample))]      
                           
                           #vergleich length first


#     for i in range(len(gtsents)):
#          for j in range(len(gtsents[i])):
#             gttok = gtsents[i][j]
#             print(gttok)
#             anontok = anonsents[i][j]
#             print(anontok)
#             if is_tag(gttok) and is_tag(anontok): tp += 1
#             elif not is_tag(gttok) and is_tag(anontok): fp += 1
#             elif is_tag(gttok) and not is_tag(anontok): fn += 1
#             elif not is_tag(gttok) and not is_tag(anontok): tn += 1

    print("true positive: " + str(tp))
    print("true negative: " + str(tn))
    print("false positive: " + str(fp))
    print("false negative: " + str(fn))

    precision = tp/ (tp + fp)
    recall = tp/ (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    # for i in range(len(gttokens)):
    #     print(len(gttokens[i]))
    #     print(len(anontokens[i]))

    # tok_g = one token from ground_trouth

    print("Precision: " + str(p))
    print("Recall: " + str(r))
    print("F1: " + str(f1))
#         for node in ne_parsetree:
#             ner_tag = str(node)[1:4]
#             if isinstance(node, nltk.tree.Tree) and (ner_tag in anonymize_tag_list):
#                 anon = ""
#                 for childnode in node:
#                     anon += '<' + ner_tag + '>'
#                 index = ne_parsetree.index(node)
#                 tokens[index] = anon
#         newline = TreebankWordDetokenizer().detokenize(tokens)
#         anonymized_text += newline + '\n'
# #     with open(new_filename, "w") as nt:
# #         f.writelines(filtered_lines)
#     # ... for every ner_tag in anonymize_tag_list replace token with ner-tag
#     return anonymized_text


In [52]:
# loaded = pickle.load(open('../models/conll2003ger', 'rb'))
# test = ConllCorpusReader('../conll2003ger', 'deu.testa', ['words', 'ignore', 'pos', 'ignore', 'chunk'])
# # def preprocessBasis(filename, new_filename):
     
# tag_list = ['PER', 'LOC']
# def basis():
#     with open(filename) as f:
#         lines = f.readlines()
#     filtered_lines = [line for line in lines if ("docstart" not in line.lower())]
#     with open(new_filename, "w") as f:
#         f.writelines(filtered_lines)   
#     # evaluation!
#     anonymized_text = ner_pii_anonymize(test, loaded, tag_list)
#     print(anonymized_text)

# g = 

model_list = [('nltkconll', '../../models/nltkconllfinal'), ('crfjoint', '../../models/finaljoint'), ('crfconll', '../../models/finalconll')]

for model in model_list:
    ner_pii_anonymize(pickle.load(open(model[1], 'rb')), anonymize_tag_list)

UnboundLocalError: local variable 'model_index' referenced before assignment

In [25]:

textpath = '../test/basis.test'

# plaintext = PlaintextCorpusReader('../test', 'basis.test')

a_text = ner_pii_anonymize(textpath, ner_tagger, anonymize_tag_list)

print(a_text)
# entity from two words
        
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[0], tuple))
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[0], nltk.tree.Tree))
#     if isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1], nltk.tree.Tree):
#         loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1].show()
#         print(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1])
#         print(str(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1])[1:4])
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1], tuple))
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[1], nltk.tree.Tree))
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[2], tuple))
#     print(isinstance(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[2], nltk.tree.Tree))
#     print()
#     print(loaded.parse(pos_tagger.tag(nltk.tokenize.WordPunctTokenizer().tokenize(line)))[3])
# textpath = "Clemens Jäger entfuhr (CDU) in diesem Zusammenhang die bemerkenswerte Äußerung, Übermäßiger Wasserverbrauch müsse künftig wieder verzichtbar sein, was Karl-Heinz Schönberg (SPD) zu der ebenso fundierten Replik veranlaßte, man müsse mit Wasser sparsam umgehen."
# ano_text = ner_pii_anonymize(textpath, fileid, loaded, tag_list)
# print(ano_text)

NameError: name 'ner_tagger' is not defined

true positive: 13
true negative: 415
false positive: 8
false negative: 8
Precision: 0.6190476190476191
Recall: 0.6190476190476191
F1: 0.6190476190476191


In [27]:
# import pandas as pd
# data = pd.read_csv(, encoding= 'unicode_escape')
# data.to_csv('../../data/conll2003/deu.utf.testb', index = False)

with open('../../data/conll2003/anon_testdata', encoding = "ISO-8859-1") as f:
    dt = f.read()
with open('../../data/conll2003/anon_testdata.fixed') as fn:
    fn.write(dt)

UnsupportedOperation: not writable

In [40]:
# loaded = pickle.load(open('../models/conll2003ger', 'rb'))
# test = ConllCorpusReader('../conll2003ger', 'deu.testa', ['words', 'ignore', 'pos', 'ignore', 'chunk'])
# # def preprocessBasis(filename, new_filename):
     
# tag_list = ['PER', 'LOC']
# def basis():
#     with open(filename) as f:
#         lines = f.readlines()
#     filtered_lines = [line for line in lines if ("docstart" not in line.lower())]
#     with open(new_filename, "w") as f:
#         f.writelines(filtered_lines)   
#     # evaluation!
#     anonymized_text = ner_pii_anonymize(test, loaded, tag_list)
#     print(anonymized_text)



UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte